In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
import pandas as pd
import numpy as np
from datetime import datetime
from difflib import SequenceMatcher

from betting.data.sportde import set_dtypes

# Processing the kaggle dataset 

## Filetype

In [3]:
data_dir = Path('../../../data/kaggle')
csv_path = data_dir / 'closing_odds.csv'
target_path = data_dir / 'closing_odds.feather'

In [4]:
if not target_path.is_file():
    df = pd.read_csv(csv_path)
    df.sort_values('match_date', inplace=True)
    df = df.reset_index(drop=True)

    df.drop(['top_bookie_home_win', 'top_bookie_draw', 'top_bookie_away_win', 'n_odds_home_win', 'n_odds_draw', 'n_odds_away_win'], inplace=True, axis=1)

    df = set_dtypes(df, [int, str, str, str, int, str, int, float, float, float, float, float, float])
    df.match_date = pd.to_datetime(df.match_date, format='%Y-%m-%d')

    df.to_feather(target_path)

df = pd.read_feather(target_path)

In [5]:
df.columns, df.shape

(Index(['match_id', 'league', 'match_date', 'home_team', 'home_score',
        'away_team', 'away_score', 'avg_odds_home_win', 'avg_odds_draw',
        'avg_odds_away_win', 'max_odds_home_win', 'max_odds_draw',
        'max_odds_away_win'],
       dtype='object'),
 (479440, 13))

## Compare against meta

In [6]:
meta_path = Path('../../../data/sportde/bundesliga')
matches_df = pd.read_feather(meta_path/'games.feather')
matches_df.head(10)

,season,matchday,home_team,away_team,home_goals,away_goals,league
0,0506,1,FC Bayern,M'gladbach,3,0,bundesliga
1,0506,1,Hamburger SV,Nürnberg,3,0,bundesliga
2,0506,1,1.FC Köln,Mainz 05,1,0,bundesliga
3,0506,1,Duisburg,VfB Stuttgart,1,1,bundesliga
4,0506,1,Wolfsburg,Dortmund,2,2,bundesliga
5,0506,1,Hannover 96,Hertha BSC,2,2,bundesliga
6,0506,1,SV Werder,Bielefeld,5,2,bundesliga
7,0506,1,Schalke 04,K'lautern,2,1,bundesliga
8,0506,1,Frankfurt,Leverkusen,1,4,bundesliga
9,0506,2,Hertha BSC,Frankfurt,2,0,bundesliga


## Split league

In [7]:
buli_str = 'Germany: Bundesliga'
buli_df = df.loc[df.league==buli_str,:].copy()
buli_df.league = 'bundesliga'

In [8]:
buli_df.tail()

,match_id,league,match_date,home_team,home_score,away_team,away_score,avg_odds_home_win,avg_odds_draw,avg_odds_away_win,max_odds_home_win,max_odds_draw,max_odds_away_win
474730,870166,bundesliga,2015-05-23,Dortmund,3,Werder Bremen,2,1.2541,6.4234,9.9490,1.30,7.47,13.00
474731,870164,bundesliga,2015-05-23,Bayern Munich,2,Mainz,0,1.1700,7.9562,13.4545,1.20,9.48,17.00
474739,870165,bundesliga,2015-05-23,B. Monchengladbach,1,Augsburg,3,1.4703,4.5645,6.3879,1.53,5.00,7.16
475481,871556,bundesliga,2015-05-28,Hamburger SV,1,Karlsruher,1,2.2679,3.1768,3.2704,2.46,3.40,3.85
476386,872810,bundesliga,2015-06-01,Karlsruher,1,Hamburger SV,1,2.1682,3.3089,3.3564,2.30,3.45,3.87


In [9]:
len(buli_df)

2747

## Seasons

In [10]:
row = buli_df.iloc[0,:]
date = row.match_date
date.year, date.month

(2005, 1)

In [11]:
def season_from_date(row):
    date = row.match_date
    if date.month<=7:
        season = (date.year-1)%100
    else:
        season = (date.year)%100

    return season

In [12]:
buli_df['season'] = buli_df.apply(func=season_from_date, axis=1)

In [13]:
buli_df.tail()

,match_id,league,match_date,home_team,home_score,away_team,away_score,avg_odds_home_win,avg_odds_draw,avg_odds_away_win,max_odds_home_win,max_odds_draw,max_odds_away_win,season
474730,870166,bundesliga,2015-05-23,Dortmund,3,Werder Bremen,2,1.2541,6.4234,9.9490,1.30,7.47,13.00,14
474731,870164,bundesliga,2015-05-23,Bayern Munich,2,Mainz,0,1.1700,7.9562,13.4545,1.20,9.48,17.00,14
474739,870165,bundesliga,2015-05-23,B. Monchengladbach,1,Augsburg,3,1.4703,4.5645,6.3879,1.53,5.00,7.16,14
475481,871556,bundesliga,2015-05-28,Hamburger SV,1,Karlsruher,1,2.2679,3.1768,3.2704,2.46,3.40,3.85,14
476386,872810,bundesliga,2015-06-01,Karlsruher,1,Hamburger SV,1,2.1682,3.3089,3.3564,2.30,3.45,3.87,14


In [14]:
for season,group in buli_df.groupby('season'):
    print(season, len(group))

4 153
5 272
6 306
7 306
8 274
9 272
11 272
12 308
13 308
14 276


## Team names

In [15]:
standings = pd.read_feather(meta_path/'standings.feather')
standings.head()

,season,matchday,rank,team,games_played,wins,draw,lost,total_goals_scored,total_goals_received,points,league
0,0506,1,1,SV Werder,1,1,0,0,5,2,3,bundesliga
1,0506,1,2,Leverkusen,1,1,0,0,4,1,3,bundesliga
2,0506,1,3,FC Bayern,1,1,0,0,3,0,3,bundesliga
3,0506,1,3,Hamburger SV,1,1,0,0,3,0,3,bundesliga
4,0506,1,5,Schalke 04,1,1,0,0,2,1,3,bundesliga


In [16]:
league = 'bundesliga'
league_standings = standings.loc[standings.league==league, :].copy()
target_teams = list(league_standings.team.unique())
len(target_teams)

35

In [17]:
actual_teams = list(buli_df.home_team.unique())
len(actual_teams)

30

In [18]:
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [19]:
translation = {}

for actual_team in actual_teams:
    scores = np.array([similar(actual_team, target_team) for target_team in target_teams])
    max_id = scores.argmax()
    translation[actual_team] = target_teams[max_id]

translation

{'Bayern Munich': 'FC Bayern',
 'Wolfsburg': 'Wolfsburg',
 'Schalke': 'Schalke 04',
 'Nurnberg': 'Nürnberg',
 'Mainz': 'Mainz 05',
 'Hansa Rostock': 'Rostock',
 'B. Monchengladbach': "M'gladbach",
 'Bochum': 'Bochum',
 'Hannover': 'Hannover 96',
 'Kaiserslautern': "K'lautern",
 'VfB Stuttgart': 'VfB Stuttgart',
 'Hamburger SV': 'Hamburger SV',
 'Freiburg': 'SC Freiburg',
 'Bayer Leverkusen': 'Leverkusen',
 'Dortmund': 'Dortmund',
 'Arminia Bielefeld': 'Bielefeld',
 'Hertha Berlin': 'Hertha BSC',
 'Werder Bremen': 'SV Werder',
 'Duisburg': 'Duisburg',
 'Eintracht Frankfurt': 'Frankfurt',
 'Alemannia Aachen': 'Aachen',
 'Energie Cottbus': 'Cottbus',
 'Karlsruher': 'Karlsruhe',
 'Hoffenheim': 'Hoffenheim',
 'Augsburg': 'Augsburg',
 'Greuther Furth': 'Gr. Fürth',
 'Dusseldorf': 'Düsseldorf',
 'Braunschweig': 'Braunschweig',
 'Paderborn': 'Paderborn',
 'Köln': '1.FC Köln'}

In [20]:
buli_df['home_team'].replace(translation, inplace=True)
buli_df['away_team'].replace(translation, inplace=True)

In [21]:
buli_df.head()

,match_id,league,match_date,home_team,home_score,away_team,away_score,avg_odds_home_win,avg_odds_draw,avg_odds_away_win,max_odds_home_win,max_odds_draw,max_odds_away_win,season
616,171624,bundesliga,2005-01-21,FC Bayern,3,Hamburger SV,0,1.421,3.846,6.840,1.45,4.00,7.75,4
644,171806,bundesliga,2005-01-22,Wolfsburg,1,Dortmund,2,1.838,3.325,3.785,1.91,3.45,4.20,4
645,171805,bundesliga,2005-01-22,Schalke 04,2,SV Werder,1,1.996,3.245,3.360,2.13,3.35,3.75,4
646,171804,bundesliga,2005-01-22,Nürnberg,1,K'lautern,3,1.957,3.270,3.435,2.00,3.45,3.75,4
647,171803,bundesliga,2005-01-22,Mainz 05,2,VfB Stuttgart,3,3.061,3.217,2.128,3.30,3.30,2.20,4


## Matchday

In [22]:
seasons = matches_df.season.unique()
season_translation = {season: int(season[:2]) for season in seasons}

matches_df.season.replace(season_translation, inplace=True)
matches_df.rename(columns={'home_goals':'home_score', 'away_goals':'away_score'}, inplace=True)
matches_df.head()

,season,matchday,home_team,away_team,home_score,away_score,league
0,5,1,FC Bayern,M'gladbach,3,0,bundesliga
1,5,1,Hamburger SV,Nürnberg,3,0,bundesliga
2,5,1,1.FC Köln,Mainz 05,1,0,bundesliga
3,5,1,Duisburg,VfB Stuttgart,1,1,bundesliga
4,5,1,Wolfsburg,Dortmund,2,2,bundesliga


In [34]:
 len(buli_df), len(matches_df)

(2747, 4896)

In [35]:
buli_df = pd.merge(buli_df, matches_df, on=['season', 'league', 'home_team', 'away_team', 'home_score', 'away_score'])
buli_df.tail()

,match_id,league,match_date,home_team,home_score,away_team,away_score,avg_odds_home_win,avg_odds_draw,avg_odds_away_win,max_odds_home_win,max_odds_draw,max_odds_away_win,season,matchday
2580,870168,bundesliga,2015-05-23,Hamburger SV,2,Schalke 04,0,2.1193,3.6983,3.1952,2.25,3.90,3.54,14,34
2581,870167,bundesliga,2015-05-23,Frankfurt,2,Leverkusen,1,4.2062,4.0383,1.7552,4.50,4.27,1.84,14,34
2582,870166,bundesliga,2015-05-23,Dortmund,3,SV Werder,2,1.2541,6.4234,9.9490,1.30,7.47,13.00,14,34
2583,870164,bundesliga,2015-05-23,FC Bayern,2,Mainz 05,0,1.1700,7.9562,13.4545,1.20,9.48,17.00,14,34
2584,870165,bundesliga,2015-05-23,M'gladbach,1,Augsburg,3,1.4703,4.5645,6.3879,1.53,5.00,7.16,14,34


In [36]:
len(result)

2585

## Standing

In [39]:
join_standings = standings.copy()
join_standings.matchday += 1
join_standings.season.replace(season_translation, inplace=True)

join_standings.head()

,season,matchday,rank,team,games_played,wins,draw,lost,total_goals_scored,total_goals_received,points,league
0,5,2,1,SV Werder,1,1,0,0,5,2,3,bundesliga
1,5,2,2,Leverkusen,1,1,0,0,4,1,3,bundesliga
2,5,2,3,FC Bayern,1,1,0,0,3,0,3,bundesliga
3,5,2,3,Hamburger SV,1,1,0,0,3,0,3,bundesliga
4,5,2,5,Schalke 04,1,1,0,0,2,1,3,bundesliga


In [40]:
joined = pd.merge(buli_df, join_standings, left_on=['home_team', 'matchday', 'season', 'league'], right_on=['team', 'matchday', 'season', 'league'], how='inner')
joined.head()

,match_id,league,match_date,home_team,home_score,away_team,away_score,avg_odds_home_win,avg_odds_draw,avg_odds_away_win,...,matchday,rank,team,games_played,wins,draw,lost,total_goals_scored,total_goals_received,points
0,195980,bundesliga,2005-08-13,K'lautern,5,Duisburg,3,1.7756,3.3456,4.1000,...,2,13,K'lautern,1,0,0,1,1,2,0
1,195981,bundesliga,2005-08-13,Nürnberg,1,Hannover 96,1,2.1989,3.2356,2.9011,...,2,17,Nürnberg,1,0,0,1,0,3,0
2,195975,bundesliga,2005-08-13,Bielefeld,0,Hamburger SV,2,3.0800,3.2850,2.1150,...,2,15,Bielefeld,1,0,0,1,2,5,0
3,195976,bundesliga,2005-08-13,Leverkusen,2,FC Bayern,5,2.8970,3.2640,2.2400,...,2,2,Leverkusen,1,1,0,0,4,1,3
4,195977,bundesliga,2005-08-13,M'gladbach,1,Wolfsburg,1,2.1356,3.2400,3.0456,...,2,17,M'gladbach,1,0,0,1,0,3,0


In [ ]:
joined.tail()

In [ ]:
joined['position_diff'] = joined.rank

In [ ]:
matchday_mask = (5<joined.matchday)
